<div style="max-width:1400px;margin-center: auto">
<img src="images\driver Standing.png" width="600"/>
</div>

WIN is calculated as sum of times where driver was in 1st position

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("Driver Standings") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

In [2]:
race_result = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\presentation\race_result')
race_result.show(2)

+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
|race_year|        race_name|          race_date|circuit_location|   driver_name|driver_number|driver_nationality|   team|grid|fastest_lap|race_time|points|position|
+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
|     1974|German Grand Prix|1974-08-04 00:00:00|         Nürburg|Clay Regazzoni|         null|             Swiss|Ferrari|   2|       null|1:41:35.0|   9.0|       1|
|     1974|German Grand Prix|1974-08-04 00:00:00|         Nürburg|Jody Scheckter|         null|     South African|Tyrrell|   4|       null|    +50.7|   6.0|       2|
+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
only

In [3]:
from pyspark.sql.functions import sum, when, desc ,rank, count
from pyspark.sql.window import Window

In [4]:
driver_standing = race_result.\
                    groupBy('driver_nationality', 'driver_name', 'race_year', 'team').\
                    agg(sum('points').alias('Total Points'),
                    count(when(race_result.position == 1, True)).alias('wins'))

In [5]:
driverRankSpec = Window.partitionBy('race_year').orderBy(desc('Total Points'), desc('wins'))
Driver_standings = driver_standing.withColumn('rank', rank().over(driverRankSpec))
Driver_standings.filter('race_year = 2020').show(11)

+------------------+----------------+---------+------------+------------+----+----+
|driver_nationality|     driver_name|race_year|        team|Total Points|wins|rank|
+------------------+----------------+---------+------------+------------+----+----+
|           British|  Lewis Hamilton|     2020|    Mercedes|       347.0|  11|   1|
|           Finnish| Valtteri Bottas|     2020|    Mercedes|       223.0|   2|   2|
|             Dutch|  Max Verstappen|     2020|    Red Bull|       214.0|   2|   3|
|           Mexican|    Sergio Pérez|     2020|Racing Point|       125.0|   1|   4|
|        Australian|Daniel Ricciardo|     2020|     Renault|       119.0|   0|   5|
|           Spanish|    Carlos Sainz|     2020|     McLaren|       105.0|   0|   6|
|              Thai| Alexander Albon|     2020|    Red Bull|       105.0|   0|   6|
|        Monegasque| Charles Leclerc|     2020|     Ferrari|        98.0|   0|   8|
|           British|    Lando Norris|     2020|     McLaren|        97.0|   

In [6]:
Driver_standings.write.format('parquet').saveAsTable('f1_presentation.driver_standings')